In [26]:
# 南璃 字段内容explode,转成多行
def concatBarcodes(path):
    import pandas as pd
    import openpyxl
    # /Users/huangjiyong/Desktop/大佬帮帮忙~.xlsx
    tab = pd.read_excel(path)
    tab['颜色'] = tab['颜色'].map(lambda x: x.split(','))
    tab['规格'] = tab['规格'].map(lambda x: x.split(','))
    tab = tab.explode('颜色') .explode('规格')
    tab['条码'] = tab['商品'] + tab['颜色'] + tab['规格']
    tab = tab.reset_index(drop=True)
    writer = pd.ExcelWriter(path,engine = 'openpyxl') 
    book = openpyxl.load_workbook(writer.path)
    writer.book = book
    tab.to_excel(writer,index=False,sheet_name='条码')
    writer.save()
    writer.close()
    return tab.shape
concatBarcodes('/Users/huangjiyong/Desktop/大佬帮帮忙~.xlsx')

(45, 44)

In [45]:
import pandas as pd
import random
shop = pd.read_excel('shop_info.xlsx',header=None)
data_time = ['2021-11-10', '2021-11-11']
list_ = []
for hour in range(1,25):
    for value in shop.itertuples():
        data = [random.sample(data_time, 1)[0], hour, value[1:][1], value[1:][2], value[1:][0], 
                random.randint(100 * hour, 1000 * hour), random.randint(10 * hour, 1000 * hour), 
                random.uniform(1000 * hour, 100000 * hour), 
                'root','2021-11-04 15:53:16.068785000','2021-11-04 15:53:16.068785000',
                random.randint(10 * hour, 1000 * hour),random.uniform(10 * hour, 200 * hour), random.randint(10 * hour, 1000 * hour)
                ]
        list_.append(data)
tab = pd.DataFrame(list_,columns=['data_time', 'data_hours', 'plat', 'shop_name','brand', 
'sales', 'order_num',' transaction_amount', 'user', 'create_time', 'update_time','unique_visitor','avg_transaction_val','customer_num'])


(648, 14)

## 日期序列

In [138]:
import math, time, random, requests, json, datetime
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

def dateRange(beginDate, endDate):
    dates = []
    dt = datetime.datetime.strptime(beginDate, "%Y-%m-%d")
    date = beginDate[:]
    while date <= endDate:
        dates.append(date)
        dt = dt + relativedelta(days=1)
        date = dt.strftime("%Y-%m-%d")
    return dates

## 获取天猫订单

In [86]:
days = dateRange('2021-11-01','2021-11-14')

def parse(header, day,page):

    header = header
    params = (
        ('page', page),
        ('pageSize', '100'),
        ('dateType', 'day'),
        ('dateRange', '{}|{}'.format(day, day)),
        ('_', '1637025406340'),
        ('token', '0d6e57571'),
    )

    response = requests.get('https://sycm.taobao.com/fa/trade/tradeDetail.json', headers=headers, params=params)
    data = json.loads(str(response.content,'utf-8'))['data']
    return data

def get_json_data(days):
    result = []
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
            'cookie': 'enc=yIUTZTeWEu5lp6cbGqEOuUoaHrrQ5oBoPdVJabUHMNHhjZWVKosJ3bmlbWQbdXkyZNa5FG8%2FI7VPtvemKdQ8Vg%3D%3D; thw=cn; t=53e19c1df9706126d7df4b8a39053717; _tb_token_=IRcXLOvcT7zOCuTHZuU4; _samesite_flag_=true; cookie2=1326c42d421b0a22b67d4db93dde353d; xlly_s=1; cancelledSubSites=empty; _m_h5_tk=7f44ba0212ae31c594da32f478b38a50_1636971796505; _m_h5_tk_enc=ca7f7d7451251084f8f1f1b1a18b916c; XSRF-TOKEN=4f78334b-299d-4055-81ff-2d4722dd1fb4; sgcookie=E100oRBfKV9YXvCkcShcdEFNCJiET8kvqeJs955KNSDyJ%2Fs48Ftb5o9qeLi0y1jw0BFjkA1jwf6n4P3XZQTkKFz7mg%3D%3D; uc1=cookie14=Uoe3cO3WAwWBSQ%3D%3D&cookie21=V32FPkk%2FhSg%2F; unb=2212779745505; sn=burandoeno%E6%97%97%E8%88%B0%E5%BA%97%3Aray; csg=747e0fe4; skt=fbe9627b9bd3ddb1; _cc_=W5iHLLyFfA%3D%3D; _euacm_ac_l_uid_=2212779745505; 2212779745505_euacm_ac_c_uid_=1881680310; 2212779745505_euacm_ac_rs_uid_=1881680310; v=0; cna=kQvXGBS4RTECAToiIxw1oXsJ; _euacm_ac_rs_sid_=107459287; JSESSIONID=50D05780B09C048A4AE9CBD1AB72A2D4; tfstk=cychB6GpklsfMreihBNIWoUCVTHhZXfzmbls_jzsoukjsjGNit6N3-RWSkxXYT1..; l=fBOs-Q9rgm7U7_zMBOfZFurza77OSIRxjuPzaNbMi95POG5k5S2GW6BcFEYDC3GVFsYBR3WLw4aLBeYBcIAreYFT5g9-R2MmnmOk-Wf..; isg=BMbGp5qk20IKpY--2kkH59bEF7pIJwrhPbMcHrDvsunAs2bNGLda8axFi-9_KAL5',
        }

    for day in days:
        
        data_1 = parse(header=header, day=day, page=1)
        result.append(data_1)
        pages = math.ceil(data_1.get('recordCount')/100)
        time.sleep(random.uniform(3,8))
        for page in range(2,pages+1):
            data = parse(header=header, day=day, page=page)
            time.sleep(random.uniform(3,8))
            result.append(data)
    return result

def get_order_info():
    Eno = []
    result = get_json_data(days)
    for detail in result:
        orders = detail.get('data')
        for order in orders:
            orderId = order.setdefault('orderId','-')

            payFee = order.setdefault('payFee',-9999)
            confirmFee = order.setdefault('confirmFee',-9999)

            payTime = order.setdefault('payTime','-')
            gmtCreateTime = order.setdefault('gmtCreateTime','-')
            Eno.append([orderId, payFee, confirmFee, payTime, gmtCreateTime])
    tab = pd.DataFrame(Eno, columns=['订单编号','支付金额', '确认收货金额','支付时间', '订单创建时间'])
    return tab

## oms订单和天猫后台订单对比

In [95]:
tab = get_order_info()
oms_path = "/Users/huangjiyong/Desktop/海澜/文档/云雀OMS/表格/订单明细报表_20211101-20211114.xlsx"
oms = pd.read_excel(oms_path)
oms_orderId = oms[['平台单号']].drop_duplicates(keep='first', inplace=False)
list_oms_orderId = []
for index, value in oms_orderId.itertuples():
    if ',' in value:
        split_ids = value.split(',')
        for split_id in split_ids:
            list_oms_orderId.append([value, split_id])
    else:
        list_oms_orderId.append([value, value])

pd_oms_orderId = pd.DataFrame(list_oms_orderId, columns=['oms平台单号','订单编号'])
pd_merge = pd.merge(tab[['订单编号','订单创建时间']], pd_oms_orderId, how='outer', on='订单编号')

In [144]:
tab.loc[(tab['支付金额']>100) & (tab['支付金额']<500)]

,订单编号,支付金额,确认收货金额,支付时间,订单创建时间
0,2221166990146676433,322.69,322.69,2021-11-01 23:22:33,2021-11-01 23:21:04
2,2069106652946890151,258.65,258.65,2021-11-01 22:59:41,2021-11-01 22:59:03
3,2220743161983071173,157.94,157.94,2021-11-01 22:50:30,2021-11-01 22:50:26
4,2069054524474137211,175.00,0.00,2021-11-01 22:51:20,2021-11-01 22:49:40
5,2221866975549918440,310.62,310.62,2021-11-01 22:47:40,2021-11-01 22:47:29
...,...,...,...,...,...
913,2250770077885165062,163.95,0.00,2021-11-14 10:49:12,2021-11-14 10:49:05
914,2251091126027213033,270.70,0.00,2021-11-14 10:36:35,2021-11-14 10:36:29
917,1439606679876818398,391.60,0.00,2021-11-14 09:04:44,2021-11-14 09:04:39
918,2249215668074643965,241.53,0.00,2021-11-14 08:06:06,2021-11-14 08:05:56
